# Part 3 - Cleaning the data ready for analysis 

### First task: Consolidate the data collected in part 2) and cleaning it.

- Data was collected in 5 separate runs for the top 5 manufacturers. The resulting csv files are all in the same format and will need to follow the same cleaning/formatting process, so they need to be consolidated in to a single dataframe. Then:


- The **title** field needs to be parsed in to name/manufacturer, and year fields
- The **mileage** and **asking price** need to be converted from strings in to numbers
- Engine details need to be split to 3 fields: **capacity**, **diesel/petrol** binary, and **manual/automatic/unspecified gearbox**
- **Vendor type** needs to be a binary dealer/private vendor
- In order to compare make/model between locations, they need to be grouped by important characteristics, eg mileage, age, engine size. These are all subtly different and input by users manually, so need to be grouped in to 'bins' for each category (e.g. "less than 100km" instead of "8km").



In [1]:
# Import necessary packages
import pandas as pd
import numpy as np

#### Import, consolidate, and check data:

In [2]:
# NB sep (separator) explicitly set to '\t' to avoid error of pandas 'seeing' more columns than there are.
ford = pd.read_csv('csv_files/ford.tsv', sep='\t')
mazda = pd.read_csv('csv_files/mazda.tsv', sep='\t')
mitsubishi = pd.read_csv('csv_files/mitsubishi.tsv', sep='\t')
nissan = pd.read_csv('csv_files/nissan.tsv', sep='\t')
toyota = pd.read_csv('csv_files/toyota.tsv', sep='\t')

In [3]:
# Consolidate individual dataframes to individual dataframe
all_df = pd.concat([ford, mazda, mitsubishi, nissan, toyota])  

In [4]:
# Show all columns and check format of resulting dataframe:
pd.set_option('display.max_columns', None)
all_df.head()

,listing_id,location,title,sub_title,mileage,engine_details,asking_price,vendor,sale_type
0,1668261987118-3860157887-location,"Napier, Hawke's Bay",2019 Ford Ranger,Ask us about Finance options on all vehicles!,"85,796km","3200cc, Diesel, Tiptronic","$53,990",NaN,"Asking price $53,990"
1,1668261987204-3860151293-location,"Whakatane, Bay of Plenty",1980 Ford Cortina,1980 Cortina Ghia,"164,600km","2000cc, Petrol, Automatic","$11,000",Private seller,sale_type missing
2,1668261987215-3860060631-location,"Kerikeri, Northland",2006 Ford Courier,sub_title missing,"321,862km","2606cc, Petrol, Manual","$6,900",Private seller,sale_type missing
3,1668261987310-3860015900-location,"Porirua, Wellington",1975 Ford Falcon,sub_title missing,96km,"4100cc, Petrol, Manual","$45,000",Private seller,"Asking price $45,000"
4,1668261987400-3859647210-location,"Cromwell, Otago",2011 Ford Falcon,Fabulous to drive,"215,856km","3984cc, Petrol, Automatic","$9,800",Private seller,sale_type missing


In [5]:
# Verify all lines have been included in the consolidated dataframe:
print(f"Ford = {ford.shape[0]} lines")
print(f"Mazda = {mazda.shape[0]} lines")
print(f"Mitsubishi = {mitsubishi.shape[0]} lines")
print(f"Nissan = {nissan.shape[0]} lines")
print(f"Toyota = {toyota.shape[0]} lines")
print(f"Consolidated df confirmed as {all_df.shape[0]} lines") 

Ford = 4026 lines
Mazda = 6467 lines
Mitsubishi = 4004 lines
Nissan = 6930 lines
Toyota = 14674 lines
Consolidated df confirmed as 36101 lines


#### Format data - Year and age:

In [6]:
# Separate year from title field:
all_df['Year'] = all_df['title'].str[:4]

In [7]:
# Create column calculating age:
all_df['Age'] = all_df['Year'] 
all_df['Age'] = 2023 - all_df['Age'].astype(int)

In [8]:
# Examine statistical analysis of ages to set appropriate bins:
all_df['Age'].describe()

count    36101.000000
mean        10.817567
std          8.299382
min          0.000000
25%          6.000000
50%          9.000000
75%         14.000000
max        135.000000
Name: Age, dtype: float64

In [9]:
# Note anomaly of 135 year old car; investigate:
filt = all_df['Age'] == 135
all_df[filt]

,listing_id,location,title,sub_title,mileage,engine_details,asking_price,vendor,sale_type,Year,Age
11802,1668271635063-3816940931-location,"Whangarei, Northland",1888 Toyota Hilux,sub_title missing,mileage missing,"3800cc, Automatic","$5,500",Private seller,"Or near offer $5,500",1888,135


In [10]:
# Hilux obviously a typo; change year from 1888 to 1988:
all_df.loc[filt, "Year"] = "1988"

# Recalculate ages:
all_df['Age'] = all_df['Year'] 
all_df['Age'] = 2023 - all_df['Age'].astype(int)

# Check result:
all_df.loc[filt]

,listing_id,location,title,sub_title,mileage,engine_details,asking_price,vendor,sale_type,Year,Age
11802,1668271635063-3816940931-location,"Whangarei, Northland",1888 Toyota Hilux,sub_title missing,mileage missing,"3800cc, Automatic","$5,500",Private seller,"Or near offer $5,500",1988,35


In [11]:
# Oldest car of 100 years old is plausible for a classic; no further investigation at this stage.
all_df['Age'].describe()

count    36101.000000
mean        10.814797
std          8.274584
min          0.000000
25%          6.000000
50%          9.000000
75%         14.000000
max        100.000000
Name: Age, dtype: float64

In [12]:
# Sort mileage in to bins to allow models of similar condition to be grouped - max age at end of 2023:

bins = [0,
        1, 
        2, 
        3, 
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        15,
        20
        ]

names = ['New',
        'New',
        '2 to 3 years old', 
        '3 to 4 years old', 
        '4 to 5 years old', 
        '5 to 6 years old',         
        '6 to 7 years old', 
        '7 to 8 years old', 
        '8 to 9 years old',
        '9 to 10 years old',
        '10 to 15 years old',
        '15 to 20 years old',
        # '1 to 20 years old',
        'Over 20 years'
        ]

d = dict(enumerate(names, 1))
all_df['age_bin'] = np.vectorize(d.get)(np.digitize(all_df['Age'], bins))

#### Format data - Make and model:

In [13]:
# Split string on first space after year
all_df['Make & Model'] = all_df['title'].str[5:]

# Drop old columns
all_df.drop(columns=['title'], inplace=True)

#### Format data - Asking price:

In [14]:
# Strip out non-numerical characters from price string; 'FutureWarning' noted:
all_df['asking_price'] = all_df['asking_price'].str.replace("$", '')
all_df['asking_price'] = all_df['asking_price'].str.replace(".", '')
all_df['asking_price'] = all_df['asking_price'].str.replace(",", '')

/tmp/ipykernel_32138/1671997852.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  all_df['asking_price'] = all_df['asking_price'].str.replace("$", '')
/tmp/ipykernel_32138/1671997852.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  all_df['asking_price'] = all_df['asking_price'].str.replace(".", '')


In [15]:
# Convert price field to float (some prices with cents noted...)
all_df['asking_price'].astype(float)

0        53990.0
1        11000.0
2         6900.0
3        45000.0
4         9800.0
          ...   
14669     3500.0
14670     4250.0
14671     6500.0
14672    11650.0
14673    15450.0
Name: asking_price, Length: 36101, dtype: float64

#### Format data - Mileage:

In [16]:
all_df['mileage_asnumber'] = all_df['mileage'].str.replace("km", "")
all_df['mileage_asnumber'] = all_df['mileage_asnumber'].str.replace("km", "")
all_df['mileage_asnumber'] = all_df['mileage_asnumber'].str.replace(",", "")
all_df['mileage_asnumber'] = all_df['mileage_asnumber'].str.replace("mileage missing", "0")

In [17]:
all_df['mileage_asnumber'] = all_df['mileage_asnumber'].astype(float)

In [18]:
# Sort mileage in to bins to allow models of similar condition to be grouped:

bins = [1, 
        100, 
        1000, 
        20000,
        50000,
        100000,
        250000,
        500000,
        ]

names = ['Less than 100km', 
        '100km to 1,000km', 
        '1,000km to 20,000km', 
        '20,000km to 50,000km', 
        '50,000km to 100,000km',
        '100,000km to 250,000km',
        '250,000km to 500,000km',
        'Over 500,000km'
        ]

d = dict(enumerate(names, 1))
all_df['mileage_bin'] = np.vectorize(d.get)(np.digitize(all_df['mileage_asnumber'], bins))

In [19]:
# Drop mileage_asnumber column:
all_df.drop(columns=['mileage_asnumber'], inplace=True)

#### Format data - Engine:

In [20]:
# Transmission - create column:
all_df['transmission'] = "Not available"

# Assign values to Automatic listings:
filt1 = all_df['engine_details'].str.contains("Automatic")
filt2 = all_df['engine_details'].str.contains("Tiptronic")
all_df.loc[filt1 | filt2, 'transmission'] = "Automatic"

# Assign values to Manual listings:
filt = all_df['engine_details'].str.contains("Manual")
all_df.loc[filt, 'transmission'] = "Manual"

In [21]:
# Fuel type - create column:
all_df['fuel'] = "Not available"

# Assign values to Petrol listings:
filt = all_df['engine_details'].str.contains("Petrol")
all_df.loc[filt, 'fuel'] = "Petrol"

# Assign values to Diesel listings:
filt = all_df['engine_details'].str.contains("Diesel")
all_df.loc[filt, 'fuel'] = "Diesel"

# Assign values to Electric listings:
filt = all_df['engine_details'].str.contains("Electric")
all_df.loc[filt, 'fuel'] = "Electric"

# Assign values to Hybrid listings:
filt1 = all_df['engine_details'].str.contains("Hybrid")
filt2 = all_df['engine_details'].str.contains("hybrid")
all_df.loc[filt1 | filt2, 'fuel'] = "Hybrid"

In [22]:
# Engine size - create column:
all_df['engine_size'] = all_df['engine_details'].str[:6]

# Filter values that do NOT contain "cc":
filt = all_df['engine_details'].str.contains("cc")
all_df.loc[~filt, 'engine_size'] = "Capacity n/a"

In [23]:
# Remove "cc" and other letters from engine size field - identified using all_df['engine_size'].unique():
all_df['engine_size'] = all_df['engine_size'].str.replace("Capacity n/a", "")
all_df['engine_size'] = all_df['engine_size'].str.replace("cc", "")
all_df['engine_size'] = all_df['engine_size'].str.replace("c", "")
all_df['engine_size'] = all_df['engine_size'].str.replace("C", "")
all_df['engine_size'] = all_df['engine_size'].str.replace("D", "")
all_df['engine_size'] = all_df['engine_size'].str.replace("P", "")
all_df['engine_size'] = all_df['engine_size'].str.replace(" ", "")
all_df['engine_size'] = all_df['engine_size'].str.replace(",", "")

In [24]:
# Fill blanks with zeros
filt = all_df['engine_size'] == ""
all_df.loc[filt, 'engine_size'] = 0

In [25]:
# Convert engine size to number and round to nearest 100cc:
all_df['engine_size'] = all_df['engine_size'].astype(int)
all_df['engine_size'] = all_df['engine_size'].round(decimals=-2)

In [26]:
# Drop engine_details column
all_df.drop(columns=['engine_details'], inplace=True)

#### Define seller type

In [27]:
# Create new column, with Dealer set as default (all that are not explicitly private sellers):
all_df['seller_type'] = "Dealer"

# Set filter:
filt = all_df['vendor'] == " Private seller "

# Apply filter and set to private seller:
all_df.loc[filt, 'seller_type'] = "Private seller"

# Drop vendor column:
all_df.drop(columns=['vendor'], inplace=True)

#### Fix locations

In [28]:
# Locations per the all_df dataframe were extracted and mapped to the desired output in the 'location.csv' file:
locations = pd.read_csv('csv_files/location.csv')
locations.head()

,location,City,Island
0,", Auckland",Auckland,North Island
1,", Canterbury",Christchurch,South Island
2,", Hawke's Bay",Other North Island,North Island
3,", Nelson Bays",Other South Island,South Island
4,", Northland",Other North Island,North Island


In [29]:
# Merge location df with all_df:
all_df = pd.merge(all_df, locations, on = "location", how = "inner")

#### Create h_string field to group models with similar key characteristics - attempt to homogenize listings

In [30]:
all_df['h_string'] = all_df['seller_type'].map(str) + "; " + all_df['age_bin'].map(str) + "; " + all_df['mileage_bin'].map(str) + "; " + all_df['engine_size'].map(str)+ "; " + all_df['fuel'].map(str) + "; " + all_df['transmission'].map(str)

#### Format final dataframe:

In [31]:
# Drop sale type - "asking price", "estimated price", and or "near offer" amount to the same thing
all_df.drop(columns=['sale_type'], inplace=True)

In [32]:
# Update column order:
all_df = all_df[['listing_id', 'Make & Model', 'location', 'City', 'Island', 'Age', 'age_bin', 'mileage', 'mileage_bin', 'engine_size', 'fuel', 'transmission', 'asking_price', 'seller_type', 'h_string', 'sub_title']]

In [33]:
# Examine final df format:
all_df.head()

,listing_id,Make & Model,location,City,Island,Age,age_bin,mileage,mileage_bin,engine_size,fuel,transmission,asking_price,seller_type,h_string,sub_title
0,1668261987118-3860157887-location,Ford Ranger,"Napier, Hawke's Bay",Other North Island,North Island,4,4 to 5 years old,"85,796km","50,000km to 100,000km",3200,Diesel,Automatic,53990,Dealer,"Dealer; 4 to 5 years old; 50,000km to 100,000k...",Ask us about Finance options on all vehicles!
1,1668261987726-3858370885-location,Ford Ranger,"Napier, Hawke's Bay",Other North Island,North Island,3,3 to 4 years old,"36,623km","20,000km to 50,000km",2000,Diesel,Automatic,69990,Dealer,"Dealer; 3 to 4 years old; 20,000km to 50,000km...",TONNEAU COVER / TOW BAR / DECK LINER
2,1668261996464-3856779556-location,Ford Everest,"Napier, Hawke's Bay",Other North Island,North Island,1,New,"12,380km","1,000km to 20,000km",2000,Diesel,Automatic,74990,Dealer,"Dealer; New; 1,000km to 20,000km; 2000; Diesel...",Ask us about Finance options on all vehicles!
3,1668262051669-3837242562-location,Ford Ecosport,"Napier, Hawke's Bay",Other North Island,North Island,6,6 to 7 years old,"33,650km","20,000km to 50,000km",1500,Petrol,Automatic,15990,Dealer,"Dealer; 6 to 7 years old; 20,000km to 50,000km...",Franchise Backed-Delivery NZ Wide-Great Servi...
4,1668262068407-3820002936-location,Ford Courier,"Napier, Hawke's Bay",Other North Island,North Island,23,Over 20 years,"327,786km","250,000km to 500,000km",2500,Diesel,Manual,7250,Private seller,"Private seller; Over 20 years; 250,000km to 50...",2000 Ford Courier xlx


In [34]:
all_df.to_csv('csv_files/final_sorted_data.csv')